In [ ]:
!pip install tensorflow

In [ ]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

In [23]:
import tensorflow as tf
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.3.1
Num GPUs Available:  2


In [ ]:
!pip install transformers

In [24]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig, AutoTokenizer

max_len = 256
configuration = BertConfig()  # default parameters and configuration for BERT


In [25]:
# Save the slow pretrained tokenizer
# slow_tokenizer = BertTokenizer.from_pretrained("vinai/phobert-base")
# save_path = "phobert-base/"
# if not os.path.exists(save_path):
#     os.makedirs(save_path)
# slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer("phobert-base/vocab.txt", lowercase=True)


In [26]:
# train_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json"
# train_path = keras.utils.get_file("train.json", train_data_url)
# eval_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
# eval_path = keras.utils.get_file("eval.json", eval_data_url)
train_path = "./train_qa_1654_empty_ans.json"

In [27]:
class SquadExample:
    def __init__(self, question, context, start_char_idx, answer_text, all_answers):
        self.question = question
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.all_answers = all_answers
        self.skip = False

    def preprocess(self):
        context = self.context
        question = self.question
        answer_text = self.answer_text
        start_char_idx = self.start_char_idx

        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        answer = " ".join(str(answer_text).split())

        # Find end character index of answer in context
        end_char_idx = start_char_idx + len(answer)
        if end_char_idx >= len(context):
            self.skip = True
            return

        # Mark the character indexes in context that are in answer
        is_char_in_ans = [0] * len(context)
        for idx in range(start_char_idx, end_char_idx):
            is_char_in_ans[idx] = 1

        # Tokenize context
        tokenized_context = tokenizer.encode(context)
#         print(tokenized_context.offsets)

        # Find tokens that were created from answer characters
        ans_token_idx = []
        for idx, (start, end) in enumerate(tokenized_context.offsets):
            if sum(is_char_in_ans[start:end]) > 0:
                ans_token_idx.append(idx)
#         print(ans_token_idx)

        if len(ans_token_idx) == 0:
            self.skip = True
            return

        # Find start and end token index for tokens from answer
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]

        # Tokenize question
        tokenized_question = tokenizer.encode(question)

        # Create inputs
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
        attention_mask = [1] * len(input_ids)

        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = max_len - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
        self.context_token_to_char = tokenized_context.offsets


with open(train_path) as f:
    raw_train_data = json.load(f)
    raw_eval_data = {"data": raw_train_data["data"][600:]}
    raw_train_data = {"data": raw_train_data["data"][:600]}
#     raw_train_data = {"data": raw_train_data["data"][:10]}
#     raw_train_data = {"data": [
#                               {"title": "University_of_Notre_Dame", 
#                                "paragraphs": [
#                                                {"context": "Chelsea cần chiến thắng trước Stoke City ở Stamford Bridge để cân bằng kỷ lục 13 chiến thắng của Arsenal ở cuối mùa 2001/02. Cuối cùng, đoàn quân của HLV Conte đã hoàn thành nhiệm vụ này khi vượt qua đối thủ với tỷ số 4-2.", 
#                                                 "qas": [
#                                                         {"answers": [{"answer_start": 0, "text": "Chelsea"}, {"answer_start": 30, "text": "Stoke City"}], 
#                                                          "question": "Team?"},
#                                                         {"answers": [{"answer_start": 30, "text": "Stoke City"}, {"answer_start": 0, "text": "Chelsea"}], 
#                                                          "question": "Team?"}
#                                                         ]
#                                                }
#                                             ]
#                               }
#                               ]
#                      }

# with open(eval_path) as f:
#     raw_eval_data = json.load(f)
#     raw_eval_data = {"data": raw_eval_data["data"][:5]}\
#     raw_eval_data = {"data": [
#                               {"title": "University_of_Notre_Dame", 
#                                "paragraphs": [
#                                                {"context": "Chelsea cần chiến thắng trước Stoke City ở Stamford Bridge để cân bằng kỷ lục 13 chiến thắng của Arsenal ở cuối mùa 2001/02. Cuối cùng, đoàn quân của HLV Conte đã hoàn thành nhiệm vụ này khi vượt qua đối thủ với tỷ số 4-2.", 
#                                                 "qas": [
#                                                         {"answers": [{"answer_start": 0, "text": "Chelsea"}, {"answer_start": 30, "text": "Stoke City"}], 
#                                                          "question": "Team?"},
#                                                     {"answers": [{"answer_start": 30, "text": "Stoke City"}, {"answer_start": 0, "text": "Chelsea"}], 
#                                                          "question": "Team?"},
#                                                     {"answers": [{"answer_start": 0, "text": "Chelsea"}, {"answer_start": 30, "text": "Stoke City"}], 
#                                                          "question": "Team?"}
#                                                         ]
#                                                }
#                                             ]
#                               }
#                               ]
#                      }


def create_squad_examples(raw_data):
    squad_examples = []
    for item in raw_data["data"]:
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                question = qa["question"]
#                 print(qa)
                answer_text = qa["answers"][0]["text"]
                all_answers = [_["text"] for _ in qa["answers"]]
                start_char_idx = qa["answers"][0]["answer_start"]
                squad_eg = SquadExample(
                    question, context, start_char_idx, answer_text, all_answers
                )
                squad_eg.preprocess()
                squad_examples.append(squad_eg)
    return squad_examples


def create_inputs_targets(squad_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in squad_examples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    return x, y


train_squad_examples = create_squad_examples(raw_train_data)
x_train, y_train = create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")

eval_squad_examples = create_squad_examples(raw_eval_data)
x_eval, y_eval = create_inputs_targets(eval_squad_examples)
print(f"{len(eval_squad_examples)} evaluation points created.")
# print(x_eval)


98155 training points created.
10659 evaluation points created.


In [28]:

def create_model():
    ## BERT encoder
    encoder = TFBertModel.from_pretrained("vinai/phobert-base")

    ## QA Model
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding = encoder(
        input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
    )[0]

    start_logits = layers.Dense(1, name="start_logit", use_bias=False)(embedding)
    start_logits = layers.Flatten()(start_logits)

    end_logits = layers.Dense(1, name="end_logit", use_bias=False)(embedding)
    end_logits = layers.Flatten()(end_logits)

    start_probs = layers.Activation(keras.activations.softmax)(start_logits)
    end_probs = layers.Activation(keras.activations.softmax)(end_logits)

    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[start_probs, end_probs],
    )
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = keras.optimizers.Adam(lr=5e-5)
    model.compile(optimizer=optimizer, loss=[loss, loss])
    return model



In [29]:
use_tpu = False
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    strategy = tf.distribute.MirroredStrategy()
    # Create model
    with strategy.scope():
        model = create_model()

model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


Some layers from the model checkpoint at vinai/phobert-base were not used when initializing TFBertModel: ['roberta', 'lm_head']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['bert']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_1 (TFBertModel)   ((None, 256, 768), ( 134998272   input_4[0][0]                    
                                                                 input_6[0][0]         

In [30]:
def normalize_text(text):
    text = text.lower()

    # Remove punctuations
    exclude = set(string.punctuation)
    text = "".join(ch for ch in text if ch not in exclude)

    # Remove articles
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    text = re.sub(regex, " ", text)

    # Remove extra white space
    text = " ".join(text.split())
    return text


class ExactMatch(keras.callbacks.Callback):
    """
    Each `SquadExample` object contains the character level offsets for each token
    in its input paragraph. We use them to get back the span of text corresponding
    to the tokens between our predicted start and end tokens.
    All the ground-truth answers are also present in each `SquadExample` object.
    We calculate the percentage of data points where the span of text obtained
    from model predictions matches one of the ground-truth answers.
    """

    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self, epoch, logs=None):
        pred_start, pred_end = self.model.predict(self.x_eval)
        count = 0
        eval_examples_no_skip = [_ for _ in eval_squad_examples if _.skip == False]
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            squad_eg = eval_examples_no_skip[idx]
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):
                continue
            pred_char_start = offsets[start][0]
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]
            else:
                pred_ans = squad_eg.context[pred_char_start:]

            normalized_pred_ans = normalize_text(pred_ans)
            normalized_true_ans = [normalize_text(_) for _ in squad_eg.all_answers]
            if normalized_pred_ans in normalized_true_ans:
                count += 1
        acc = count / len(self.y_eval[0])
        print(f"\nepoch={epoch+1}, exact match score={acc:.2f}")

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="./ckpt/checkpoint",
    save_weights_only=True,
    monitor='loss',
    mode="min",
    save_best_only=True)


In [ ]:
!ls -l .

In [ ]:
exact_match_callback = ExactMatch(x_eval, y_eval)
model.fit(
    x_train,
    y_train,
    epochs=100,  # For demonstration, 3 epochs are recommended
    batch_size=8,
    callbacks=[exact_match_callback, model_checkpoint_callback],
)

Epoch 1/100
1750/2002 [=========================>....] - ETA: 1:08 - loss: 1.1236 - activation_6_loss: 0.5646 - activation_7_loss: 0.5590

In [ ]:
model.load_weights("./ckpt/checkpoint")
res = model.predict(x_eval)
print(np.array(res).shape)

idy, idx, idz = np.where(np.array(res) > 0.1)
index = np.array([idx, idy, idz])

list_answer = []
for x in set(sorted(idx)):
    ques_idx = np.where(index[0] == x)    
    start_idx = np.where(index[1] == 0)
    end_idx = np.where(index[1] == 1)

    list_start = np.intersect1d(ques_idx, start_idx)
    list_end = np.intersect1d(ques_idx, end_idx)
    
    answers = [(index[2][pair[0]], index[2][pair[1]]) for pair in zip(list_start, list_end)]
    list_answer.append(answers)
    
print(list_answer)
        

#     a = np.where(idy[ques_idx] == 0)
#     print(idy[ques_idx][a])
    
#     b = np.where(idy[ques_idx] == 1)
#     print(idy[ques_idx][b])